<a href="https://colab.research.google.com/github/AlexWortega/Collab_experement/blob/main/2chgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [1]:
!pip3 install transformers==3.5.0

     |████████████████████████████████| 1.3MB 6.8MB/s 
     |████████████████████████████████| 890kB 16.8MB/s 
     |████████████████████████████████| 2.9MB 40.4MB/s 
     |████████████████████████████████| 1.1MB 59.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=5d24ef5a21d1a8dc95b61be3b78e7f78eb4c1c0a150a906323adce54464ba422
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
pip install dvach.api

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
  Created wheel for addict: filename=addict-2.2.0-cp37-none-any.whl size=2783 sha256=d1d0a0a8e606eba5fb6d45062a4d6b840e47c1aa430f19b0a3bd52fd733419b7
  Stored in directory: /root/.cache/pip/wheels/1c/35/37/7a99ee58a72dc257689fb4f99c8018c9513555055e3fb87123
  Created wheel for simplejson: filename=simplejson-3.16.0-cp37-cp37m-linux_x86_64.whl size=114843 sha256=a71df7a2fd090d572de52854bf85536beeedd676baef7898e97b4dabd833d6c0
  Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
Successfully built addict simplejson
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.21.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: idna 2.10
 

In [2]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 595 (delta 56), reused 36 (delta 18), pack-reused 505
Receiving objects: 100% (595/595), 349.36 KiB | 4.48 MiB/s, done.
Resolving deltas: 100% (356/356), done.


In [3]:
!mkdir models/

## Download files

In [44]:
import api2ch
brds= ['b','d','sn','cc','soc','po','un']
ch = []
for b in brds:

  api = api2ch.DvachApi(b)

  board = api.get_board()

  print(f'Total {len(board)} threads in {api.board.id}')
  for thread in board:
      ch+=[(thread.post.comment)]
    


Total 208 threads in b
Total 135 threads in d
Total 191 threads in sn
Total 356 threads in cc
Total 130 threads in soc
Total 171 threads in po
Total 78 threads in un


In [45]:
len(ch)

1269

In [46]:

with open("2ch.txt", "w") as txt_file:
    for line in ch:
        txt_file.write(" ".join(line) + "\n")

## Train 
The following code download our model and tokenizer from huggingface and finetune model for generating essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/essays \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=ch.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

2021-03-05 07:18:35.933664: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
03/05/2021 07:18:38 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
03/05/2021 07:18:38 - INFO - filelock -   Lock 139621364580752 acquired on /root/.cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
Downloading: 100% 608/608 [00:00<00:00, 485kB/s]
03/05/2021 07:18:39 - INFO - filelock -   Lock 139621364580752 released on /root/.cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
03/05/2021 07:18:39 - INFO - filelock -   Lock 139621364538640 acquired on /root/.cache/torch/transformers/1b36eeb1fd7b3a6ec11bf46bde2c38e7e68f71ec774694b9e886c86001aab35d.c483bc3440

## Evaluate model
### Generate with generate_transformers.py script.

In [ ]:
!python ru-gpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=models/essays \
    --k=5 \
    --p=0.95 \
    --length=500 \
    --repetition_penalty=5

2021-02-11 12:14:52.166691: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/11/2021 12:15:04 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=5, length=500, model_name_or_path='models/essays', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.95, padding_text='', prompt='', repetition_penalty=5.0, seed=42, stop_token='</s>', temperature=1.0, xlm_language='')
Context >>> <s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: 
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение:  В данном высказывании автор поднимает проблему личности как носителя культуры.</S
Личность - это совокупность всех индивидуально значимых свойств индивидуальности («мораль», духовная культура). Личность выступает в качестве субъект

### Generate mannualy

In [5]:
import numpy as np
import torch

In [6]:
np.random.seed(42)
torch.manual_seed(42)

In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [8]:
tok = GPT2Tokenizer.from_pretrained("models/essays")

In [9]:
model = GPT2LMHeadModel.from_pretrained("models/essays")

In [ ]:
model.cuda()

In [ ]:
text = "<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: "
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), max_length=500, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)

In [ ]:
tok.decode(out[0])